In [1]:
from src.dataset.feedback_utils_v2 import Feedback
from src.dataset.format_v2 import to_dpo, to_sft, to_full, to_distill_sft
import json

feedback = Feedback(content = "Do not talk about elephant")
# sft_dataset = to_sft(feedback)
dataset = to_distill_sft(feedback)

Loaded 201 prompts
Loaded 201 search infos


In [2]:
from huggingface_hub import login
from os import getenv
# from google.colab import userdata
HF_TOKEN = getenv("HF_TOKEN")
login(
  token=HF_TOKEN, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/fangyuanyu/.cache/huggingface/token
Login successful


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import setup_chat_format
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from peft import LoraConfig
from os import getenv


# Hugging Face model id
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = 'right' # to prevent warnings


if "Meta-Llama-3-" in model_id:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
else:
    tokenizer.pad_token = tokenizer.unk_token

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

args = TrainingArguments(
    output_dir="feedback-adaptor-01", # directory to save and repository id
    num_train_epochs=3,                     # number of training epochs
    per_device_train_batch_size=1,          # batch size per device during training
    gradient_accumulation_steps=2,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    bf16=False,                              # use bfloat16 precision
    tf32=False,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    remove_unused_columns=False,
)

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [3]:
args.output_dir

'feedback-adaptor-01'

In [ ]:
from transformers import AutoTokenizer

# Hugging Face model id
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

messages = [
    {"role": "user",
     "content": "hi"},
    {"role": "assistant",
     "content": "hello"}
]
format_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
response_template = format_prompt.split("hi")[-1].split("hello")[0]

In [1]:
from src.utils_v2 import ModelArguments, PeftArguments
from transformers import HfArgumentParser, TrainingArguments, SFTTrainer
import json
from src.custom_collator import (DataCollatorForCompletionOnlyLM_v2, 
                                 get_format_func, 
                                 get_teacher_format_func,
                                 infer_response_template)
from src.dft_v2 import DFTTrainer

# Load Argument Configuration
arg_file = "configs/config_dft_v1.json"
with open(arg_file, "r") as f:
    arg_dict = json.load(f)

# Load Model
model_arg_parser = HfArgumentParser((ModelArguments,))
model_args: ModelArguments = model_arg_parser.parse_dict(arg_dict["model_args"])[0]
# model, tokenizer = model_args.make()

# Load LoRA arguments
peft_args: PeftArguments = HfArgumentParser((PeftArguments,)).parse_dict(arg_dict["lora_args"])[0]
peft_config = peft_args.make()

# Load Training Arguments
args = HfArgumentParser((TrainingArguments,)).parse_dict(arg_dict["training_args"])[0]

# Trainer Preparation
response_template = infer_response_template(tokenizer)
collator = DataCollatorForCompletionOnlyLM_v2(response_template, tokenizer=tokenizer)
formatting_prompt_func = get_format_func(tokenizer)
teacher_formatting_prompt_func = get_teacher_format_func(tokenizer)

algo = arg_dict["algorithm"]
max_seq_length = 1024

if algo == "sft":
    trainer = SFTTrainer(
        model=model,
        args=args,
        train_dataset=dataset["train"],
        peft_config=peft_config,
        max_seq_length=max_seq_length,
        tokenizer=tokenizer,
        # dataset_text_field="text", # Question: I do NOT think 'text' is one of the key in the dataset ??
        formatting_func=formatting_prompt_func,
        data_collator=collator,
        packing=False,
        dataset_kwargs={
            "add_special_tokens": False,  # We template with special tokens
            "append_concat_token": False, # No need to add additional separator token
        }
    )
elif algo == "dft":
    trainer = DFTTrainer(
        model=model,
        args=args,
        train_dataset=dataset["train"],
        peft_config=peft_config,
        max_seq_length=max_seq_length,
        tokenizer=tokenizer,
        formatting_func=formatting_prompt_func,
        student_formatting_func=formatting_prompt_func,
        teacher_formatting_func=teacher_formatting_prompt_func,
        data_collator=collator,
        response_template = response_template,
        dataset_kwargs={
            "add_special_tokens": False,  # We template with special tokens
            "append_concat_token": False, # No need to add additional separator token
        }
    )

trainer.train()

In [2]:
arg_dict["algorithm"]

'sft'

In [14]:
from src.custom_collator import (DataCollatorForCompletionOnlyLM_v2, 
                                 get_format_func, 
                                 get_teacher_format_func,
                                 infer_response_template)

response_template = infer_response_template(tokenizer)
collator = DataCollatorForCompletionOnlyLM_v2(response_template, tokenizer=tokenizer)
formatting_prompt_func = get_format_func(tokenizer)
teacher_formatting_prompt_func = get_teacher_format_func(tokenizer)

ImportError: cannot import name 'infer_response_template' from 'src.custom_collator' (/Users/fangyuanyu/Implementation/c3po/src/custom_collator.py)

In [12]:
from trl import SFTTrainer
# from src.icdft import InContextDistillTrainer
from src.dft_v2 import DFTTrainer

max_seq_length = 1024 # max sequence length for model and packing of the dataset

# This Works (!)
# trainer = SFTTrainer(
#     model=model,
#     args=args,
#     train_dataset=dataset["train"],
#     peft_config=peft_config,
#     max_seq_length=max_seq_length,
#     tokenizer=tokenizer,
#     # dataset_text_field="text", # Question: I do NOT think 'text' is one of the key in the dataset ??
#     formatting_func=formatting_prompt_func,
#     data_collator=collator,
#     packing=False,
#     dataset_kwargs={
#         "add_special_tokens": False,  # We template with special tokens
#         "append_concat_token": False, # No need to add additional separator token
#     }
# )

trainer = DFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    formatting_func=formatting_prompt_func,
    student_formatting_func=formatting_prompt_func,
    teacher_formatting_func=teacher_formatting_prompt_func,
    data_collator=collator,
    response_template = response_template,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
    }
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:487: UserWarning: You passed `remove_unused_columns=False` on a non-packed dataset. This might create some issues with the default collator and yield to errors. If you want to inspect dataset other columns (in this case ['prompt', 'completion', 'teacher_prompt']), you can subclass `DataCollatorForLanguageModeling` in case you used the default collator and create your own data collator in order to inspect the unused dataset columns.
  warnings.warn(


Map:   0%|          | 0/185 [00:00<?, ? examples/s]

Map:   0%|          | 0/185 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer.train()

/workspace/c3po/src/dft_v2.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.nn.utils.rnn.pad_sequence([torch.tensor(v) for v in example_list], batch_first=True, padding_value=pad_value)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fa

Step,Training Loss
10,3.504400
20,1.554300
30,1.585000
40,1.499000
50,1.494100
60,1.438900
